## Segmeting and Clustering Neighborhoods in Toronto

### Preprocessing the data

In [1]:
#importing libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
#Fetching data from internet(Wiki)
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautifulsoup package  to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')

In [14]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    count = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if count == 1: 
            Postcode_var = td_cell.text
        if count == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if count == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        count +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

## Integrating the codes with more than one neighbor

In [15]:
unique_p_code = set(Postcode)
print('num of unique Postal codes:', len(unique_p_code))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p_code:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 84


In [19]:
#Creating a Pandas Dataframe
toronto_dictionary = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
dataframe_toronto = pd.DataFrame.from_dict(toronto_dictionary)
dataframe_toronto.to_csv('toronto_part1.csv')
dataframe_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1N,Scarborough,Birch Cliff
2,M5P,Central Toronto,Forest Hill North
3,M1W,Scarborough,Steeles West
4,M8V,Etobicoke,New Toronto
5,M6J,West Toronto,"Little Portugal, Trinity"
6,M4L,East Toronto,India Bazaar
7,M3K,North York,CFB Toronto
8,M9M,North York,"Emery, Humberlea"
9,M6M,York,"Keelesdale, Mount Dennis, Silverthorn"


In [20]:
dataframe_toronto.shape

(84, 3)